<a href="https://colab.research.google.com/github/shufan6011/GW-Data-Analysis/blob/main/Step_3_Basic_GW_Data_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Update:
# Event Detection by DL models come in the subsequent steps

# Data Preprocessing

In [2]:
import numpy as np
import pandas as pd
import requests, os
import matplotlib.pyplot as plt
from scipy.signal import butter, filtfilt
from sklearn.preprocessing import StandardScaler

import warnings
warnings.filterwarnings('ignore')


In [3]:
# Go to https://gwosc.org
# Find info required below (GPS time & detector)


In [4]:
# Set GPS time:
t_start = 1126259462.4
t_end = 1126259462.4 # For specific events, make t_end the same as t_start

# Choose detector (H1, L1, or V1)
detector = 'H1'


In [5]:
%config InlineBackend.figure_format = 'retina'

try:
    from gwpy.timeseries import TimeSeries
except:
    ! pip install -q "gwpy==3.0.8"
    ! pip install -q "matplotlib==3.9.0"
    ! pip install -q "astropy==6.1.0"
    from gwpy.timeseries import TimeSeries


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 1.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 45.5 MB/s eta 0:00:00


In [6]:
from gwosc.locate import get_urls
url = get_urls(detector, t_start, t_end)[-1]

# If an event is chosen, then its info will be shown in url
print('Downloading: ' , url)
fn = os.path.basename(url)
with open(fn,'wb') as strainfile:
    straindata = requests.get(url)
    strainfile.write(straindata.content)


Downloading:  http://gwosc.org/eventapi/json/GWTC-1-confident/GW150914/v3/H-H1_GWOSC_4KHZ_R1-1126257415-4096.hdf5


In [7]:
# Read strain data
strain = TimeSeries.read(fn,format='hdf5.gwosc')

# Examine an interval closely
# center = int(t_start)
# strain = strain.crop(center-0.2, center+0.1)

# Extract time and strain vals
timestamps = strain.times.value
strain_values = strain.value

# Store data in pd df
data = pd.DataFrame({
    'time': timestamps,
    'strain': strain_values
})


## Handling Missing Values

In [8]:
# Drop rows with missing vals
data = data.dropna()

print("\nMissing vals after cleaning:")
print(data.isnull().sum())



Missing vals after cleaning:
time      0
strain    0
dtype: int64


## Data Noise Filtering

In [9]:
# Band-pass filter function
def butter_bandpass(lowcut, highcut, fs, order=5):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    return b, a

def bandpass_filter(data, lowcut, highcut, fs, order=5):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = filtfilt(b, a, data)
    return y

# Filter params
lowcut = 20  # Low cutoff frequency (Hz)
highcut = 500  # High cutoff frequency (Hz)

# Band-pass filter strain data
data['strain'] = bandpass_filter(data['strain'], lowcut, highcut, 4096)


## Data Normalization

In [10]:
# Normalize strain data
scaler = StandardScaler()
data['strain'] = scaler.fit_transform(data[['strain']])


## Data Inspection

In [11]:
# Inspect first few rows
print("First few rows of data:")
print(data.head())

# Inspect col headers
print("\nCol headers:")
print(data.columns)

# Summary stats
print("\nSummary stats:")
print(data.describe())

# Check for missing vals
print("\nMissing vals in each col:")
print(data.isnull().sum())

# Check sampling frequency
print(f"Sampling frequency: {strain.sample_rate} Hz")
fs = 4096 # Change this if sampling frequency is diff


First few rows of data:
           time    strain
0  1.126257e+09 -2.509170
1  1.126257e+09  0.070279
2  1.126257e+09  2.209691
3  1.126257e+09  3.618610
4  1.126257e+09  4.256309

Col headers:
Index(['time', 'strain'], dtype='object')

Summary stats:
               time        strain
count  1.677722e+07  1.677722e+07
mean   1.126259e+09 -1.758737e-17
std    1.182413e+03  1.000000e+00
min    1.126257e+09 -3.686864e+00
25%    1.126258e+09 -7.088868e-01
50%    1.126259e+09  1.167451e-03
75%    1.126260e+09  7.087773e-01
max    1.126262e+09  4.284804e+00

Missing vals in each col:
time      0
strain    0
dtype: int64
Sampling frequency: 4096.0 Hz Hz


# Time-Domain Features

In [12]:
def calc_and_print_time_domain_features(data, strain_column, fs):
    peak_amplitude = np.max(data[strain_column])
    min_amplitude = np.min(data[strain_column])
    print(f"Peak Amplitude ({strain_column}): {peak_amplitude}")
    print(f"Min Amplitude ({strain_column}): {min_amplitude}")

    threshold = 0.5 * peak_amplitude
    significant_signal = data[strain_column].abs() > threshold
    signal_duration = significant_signal.sum() * (1/fs)
    print(f"Signal Duration ({strain_column}): {signal_duration}s")

    signal_power = np.mean(data[strain_column]**2)
    noise_power = np.mean(data[data[strain_column].abs() <= threshold][strain_column]**2)
    snr = 10 * np.log10(signal_power / noise_power)
    print(f"Signal-to-Noise Ratio (SNR) ({strain_column}): {snr} dB\n")

# Calc features for strain data
print("Calc features for strain data: ")
calc_and_print_time_domain_features(data, 'strain', fs)


Calc features for strain data: 
Peak Amplitude (strain): 4.284804453733104
Min Amplitude (strain): -3.686864089465157
Signal Duration (strain): 92.680908203125s
Signal-to-Noise Ratio (SNR) (strain): 0.4926804961051281 dB



# Basic Event Detection & Parameter Estimation

In [13]:
def calc_threshold(data, strain_column, factor=3):
    noise_std = np.std(data[strain_column])
    threshold = factor * noise_std
    return threshold

def detect_events(data, strain_column, threshold):
    events = []
    event_start = None

    for i, strain in enumerate(data[strain_column]):
        if abs(strain) > threshold:
            if event_start is None:
                event_start = i
        else:
            if event_start is not None:
                event_end = i
                events.append((event_start, event_end))
                event_start = None

    # Check if an event is ongoing at end of data
    if event_start is not None:
        events.append((event_start, len(data[strain_column]) - 1))
    return events

def estimate_event_params(data, strain_column, events, fs):
    time_column = 'time'

    event_params = []
    for event in events:
        start_idx, end_idx = event
        event_data = data[strain_column].iloc[start_idx:end_idx]
        peak_amplitude = np.max(np.abs(event_data))
        duration = (end_idx - start_idx) / fs
        event_params.append({
            'start_time': data[time_column].iloc[start_idx],
            'end_time': data[time_column].iloc[end_idx - 1],
            'peak_amplitude': peak_amplitude,
            'duration': duration
        })
    return event_params

# Calc thresholds
threshold = calc_threshold(data, 'strain')

print(f"Threshold: {threshold}")

# Detect events
events = detect_events(data, 'strain', threshold)

# Estimate event params
event_params = estimate_event_params(data, 'strain', events, fs)

print("\nEvent Params:")
for param in event_params:
    print(param)


Threshold: 3.0000000000000027

Event Params:
{'start_time': 1126257415.0007324, 'end_time': 1126257415.001709, 'peak_amplitude': 4.284804453733104, 'duration': 0.001220703125}
{'start_time': 1126257418.3012695, 'end_time': 1126257418.3015137, 'peak_amplitude': 3.10656720831358, 'duration': 0.00048828125}
{'start_time': 1126257418.4667969, 'end_time': 1126257418.467041, 'peak_amplitude': 3.1396660570114685, 'duration': 0.00048828125}
{'start_time': 1126257423.3283691, 'end_time': 1126257423.3283691, 'peak_amplitude': 3.0493328722819033, 'duration': 0.000244140625}
{'start_time': 1126257423.4399414, 'end_time': 1126257423.4401855, 'peak_amplitude': 3.139458428439673, 'duration': 0.00048828125}
{'start_time': 1126257423.4418945, 'end_time': 1126257423.4421387, 'peak_amplitude': 3.1423561734113865, 'duration': 0.00048828125}
{'start_time': 1126257424.4831543, 'end_time': 1126257424.4833984, 'peak_amplitude': 3.0957766543715897, 'duration': 0.00048828125}
{'start_time': 1126257424.4851074, 

# Basic Statistical Analysis

In [14]:
def summarize_event_params(event_params):
    if not event_params:  # Check if event_params array is empty
        return {
            'num_events': 0,
            'average_duration': 0,
            'max_duration': 0,
            'average_peak_amplitude': 0,
            'max_peak_amplitude': 0
        }

    durations = [param['duration'] for param in event_params]
    peak_amplitudes = [param['peak_amplitude'] for param in event_params]

    summary = {
        'num_events': len(event_params),
        'average_duration': np.mean(durations),
        'max_duration': np.max(durations),
        'average_peak_amplitude': np.mean(peak_amplitudes),
        'max_peak_amplitude': np.max(peak_amplitudes)
    }
    return summary

# Summarize event params
summary = summarize_event_params(event_params)

print("\nSummary of Event Params:")
print(summary)


{'start_time': 1126259020.959961, 'end_time': 1126259020.959961, 'peak_amplitude': 3.077370086586475, 'duration': 0.000244140625}
{'start_time': 1126259025.7373047, 'end_time': 1126259025.7375488, 'peak_amplitude': 3.111425963748024, 'duration': 0.00048828125}
{'start_time': 1126259025.739502, 'end_time': 1126259025.7399902, 'peak_amplitude': 3.2615957267748223, 'duration': 0.000732421875}
{'start_time': 1126259032.2409668, 'end_time': 1126259032.241211, 'peak_amplitude': 3.022413962660207, 'duration': 0.00048828125}
{'start_time': 1126259033.2734375, 'end_time': 1126259033.2734375, 'peak_amplitude': 3.01910679528622, 'duration': 0.000244140625}
{'start_time': 1126259034.442871, 'end_time': 1126259034.4431152, 'peak_amplitude': 3.1782098730862822, 'duration': 0.00048828125}
{'start_time': 1126259034.6081543, 'end_time': 1126259034.6083984, 'peak_amplitude': 3.1375730792326375, 'duration': 0.00048828125}
{'start_time': 1126259035.6726074, 'end_time': 1126259035.6726074, 'peak_amplitude'